In [2]:
%pylab inline
import keras
import keras.backend as K
import kapre
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
import random
import arrow
import threading
import hyperopt
import pprint
import librosa
import samplerate
from hyperopt import hp
from soph import soph_scaler, center_wave, ex_generator
ex_df = pd.read_pickle("data/ex_df.pkl")
# https://github.com/keunwoochoi/kapre

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random', 'arrow']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
test_df = ex_df.loc[ex_df.state=="val",:].copy()
test_df.reset_index(inplace=True)
num_ex = len(test_df)
x = zeros((num_ex,1,16000))
for x_i,j in enumerate(range(num_ex)):

    x[x_i,...] = center_wave(test_df.fn[j])


In [5]:
x.shape

(7058, 1, 16000)

# Kapre-based MFCC

In [9]:
k_input = keras.layers.Input(shape=(1, 16000))
k_output = kapre.time_frequency.MFCC(
    retain=20,
    n_mfcc=80,
    n_mels=80,
    n_dft=1024,
    n_hop=256,
    power_melgram=2.0,
    return_decibel_melgram=True,
)(k_input)

k_model = keras.Model(inputs=k_input, outputs=k_output)

k_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='nadam',
    metrics=['accuracy'])

In [11]:
_ = k_model.predict(x, batch_size=512, verbose=1)

7058/7058 [==============================] - 2s 240us/step
